In [20]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{callUDF, col}
import org.datasyslab.geospark.spatialRDD.PolygonRDD
import org.datasyslab.geosparksql.utils.Adapter
import org.datasyslab.geospark.enums.{FileDataSplitter, GridType}
import org.datasyslab.geosparksql.utils.GeoSparkSQLRegistrator
import com.vividsolutions.jts.geom.Geometry

lastException: Throwable = null


In [2]:
val spark = SparkSession.builder.master("local[*]").getOrCreate()

spark = org.apache.spark.sql.SparkSession@3bcdf98f


org.apache.spark.sql.SparkSession@3bcdf98f

In [13]:
val geojsonInputLocation = "/home/pkocinski001/Desktop/projects/geo_pyspark/tests/resources/testPolygon.json"

GeoSparkSQLRegistrator.registerAll(spark)

geojsonInputLocation = /home/pkocinski001/Desktop/projects/geo_pyspark/tests/resources/testPolygon.json


lastException: Throwable = null


/home/pkocinski001/Desktop/projects/geo_pyspark/tests/resources/testPolygon.json

In [14]:
var spatialRDD = new PolygonRDD(spark.sparkContext, geojsonInputLocation, FileDataSplitter.GEOJSON, true)
spatialRDD.analyze()
var df = Adapter.toDf(spatialRDD, spark)

spatialRDD = org.datasyslab.geospark.spatialRDD.PolygonRDD@6db52195
df = [geometry: string, STATEFP: string ... 9 more fields]


[geometry: string, STATEFP: string ... 9 more fields]

In [21]:
spatialRDD.spatialPartitioning(GridType.EQUALGRID)

true

In [22]:
spatialRDD.grids.toArray.foreach(println)

Env[-88.4418791405619 : -84.984831, 30.227773279627797 : 35.013505219053194]


In [5]:
val s= spatialRDD.rawSpatialRDD.collect().toArray.take(1)(0)

s = POLYGON ((-87.621765 34.873444, -87.617535 34.873369, -87.6123 34.873337, -87.604049 34.873303, -87.604033 34.872316, -87.60415 34.867502, -87.604218 34.865687, -87.604409 34.858537, -87.604018 34.851336, -87.603716 34.844829, -87.603696 34.844307, -87.603673 34.841884, -87.60372 34.841003, -87.603879 34.838423, -87.603888 34.837682, -87.603889 34.83763, -87.613127 34.833938, -87.616451 34.832699, -87.621041 34.831431, -87.621056 34.831526, -87.62112 34.831925, -87.621603 34.8352, -87.62158 34.836087, -87.621383 34.84329, -87.621359 34.844438, -87.62129 34.846387, -87.62119 34.85053, -87.62144 34.865379, -87.621765 34.873444))	01	077	011501	5	1500000US010770115015	010770115015	5	BG	6844991	32636


POLYGON ((-87.621765 34.873444, -87.617535 34.873369, -87.6123 34.873337, -87.604049 34.873303, -87.604033 34.872316, -87.60415 34.867502, -87.604218 34.865687, -87.604409 34.858537, -87.604018 34.851336, -87.603716 34.844829, -87.603696 34.844307, -87.603673 34.841884, -87.60372 34.841003, -87.603879 34.838423, -87.603888 34.837682, -87.603889 34.83763, -87.613127 34.833938, -87.616451 34.832699, -87.621041 34.831431, -87.621056 34.831526, -87.62112 34.831925, -87.621603 34.8352, -87.62158 34.836087, -87.621383 34.84329, -87.621359 34.844438, -87.62129 34.846387, -87.62119 34.85053, -87.62144 34.865379, -87.621765 34.873444))	01	077	011501	5	1500000US010770115015	010770115015	5	BG	6844991	32636

In [6]:
import org.datasyslab.geosparksql.utils.GeometrySerializer
import org.apache.spark.sql.types._

In [7]:
val data = spatialRDD.rawSpatialRDD.rdd.map(
    data => (GeometrySerializer.serialize(data))
)

data = MapPartitionsRDD[5] at map at <console>:38


MapPartitionsRDD[5] at map at <console>:38

In [8]:
val schema = StructType(
    Array(StructField("geom", ArrayType(ByteType, containsNull=false)))
)

schema = StructType(StructField(geom,ArrayType(ByteType,false),true))


StructType(StructField(geom,ArrayType(ByteType,false),true))

In [13]:
spark.createDataFrame(data, schema)

Name: Compile Error
Message: <console>:50: error: overloaded method value createDataFrame with alternatives:
  (data: java.util.List[_],beanClass: Class[_])org.apache.spark.sql.DataFrame <and>
  (rdd: org.apache.spark.api.java.JavaRDD[_],beanClass: Class[_])org.apache.spark.sql.DataFrame <and>
  (rdd: org.apache.spark.rdd.RDD[_],beanClass: Class[_])org.apache.spark.sql.DataFrame <and>
  (rows: java.util.List[org.apache.spark.sql.Row],schema: org.apache.spark.sql.types.StructType)org.apache.spark.sql.DataFrame <and>
  (rowRDD: org.apache.spark.api.java.JavaRDD[org.apache.spark.sql.Row],schema: org.apache.spark.sql.types.StructType)org.apache.spark.sql.DataFrame <and>
  (rowRDD: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row],schema: org.apache.spark.sql.types.StructType)org.apache.spark.sql.DataFrame
 cannot be applied to (org.apache.spark.rdd.RDD[Array[Byte]], org.apache.spark.sql.types.StructType)
       spark.createDataFrame(data, schema)
             ^

StackTrace: 

MapPartitionsRDD[2] at mapPartitions at PolygonRDD.java:432

In [9]:
import org.apache.spark.api.python._

In [16]:
[PythonRDD[spark].collectAndServe(spatialRDD.rawSpatialRDD)

Name: Compile Error
Message: <console>:1: error: illegal start of definition
[PythonRDD[spark].collectAndServe(spatialRDD.rawSpatialRDD)
^

StackTrace: 

In [13]:
PythonRDDServer

Name: Compile Error
Message: <console>:36: error: not found: value PythonRDDServer
       PythonRDDServer
       ^

StackTrace: 